In [1]:
#라이브러리 import
import requests
import pprint
import json
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',50)

In [3]:
year_map = list(range(2021,2024))
month_map = ['0'+str(i) if i<10 else str(i) for i in range(1,13)]

rc_month = [f'{y}{m}' for y in year_map for m in month_map]

In [20]:
# url 입력
## 배당률 불러오기
meet = 1 # 1: 서울, 2: 제주, 3: 부산
odds = ['WIN', 'QNL','TLA']
rc_no = list(range(1, 16))

for ym in rc_month:
    for o in odds:
        for rc in rc_no:
            try:

                odds_url = f'https://apis.data.go.kr/B551015/API160_1/integratedInfo_1?serviceKey=%2FpKtn0KFociUr9CM316rFT6o8sUrRD8yjSLuICPzxXarC8O%2FrUc1pjHc%2FCpuEwB5r4%2BimnNKYuJEjPKARyZCuw%3D%3D&pageNo=1&numOfRows=10000&pool={o}&rc_month={ym}&rc_no={rc}&meet={meet}&_type=json'

                response = requests.get(odds_url)

                contents = response.text

                ## json을 DataFrame으로 변환하기 ##

                #문자열을 json으로 변경
                json_ob = json.loads(contents)

                # 필요한 내용만 꺼내기
                body = json_ob['response']['body']['items']['item']
            except Exception as e:
                print('e',e)
                continue

            # Dataframe으로 만들기
            dataframe = pd.json_normalize(body)


    chulNo  chulNo2  chulNo3 meet    odds pool    rcDate  rcNo
0        1        0        0   서울    92.8  단승식  20210108     1
1        2        0        0   서울     1.8  단승식  20210108     1
2        3        0        0   서울    92.8  단승식  20210108     1
3        4        0        0   서울    92.8  단승식  20210108     1
4        5        0        0   서울    92.8  단승식  20210108     1
5        6        0        0   서울    92.8  단승식  20210108     1
6        7        0        0   서울    92.8  단승식  20210108     1
7        8        0        0   서울     1.8  단승식  20210108     1
8        9        0        0   서울    92.8  단승식  20210108     1
9       10        0        0   서울    92.8  단승식  20210108     1
10      11        0        0   서울    92.8  단승식  20210108     1
11      12        0        0   서울    18.6  단승식  20210108     1
12       1        0        0   서울   408.8  단승식  20210115     1
13       2        0        0   서울  9999.9  단승식  20210115     1
14       3        0        0   서울  9999.9  단승식  2021011

KeyboardInterrupt: 

In [13]:
meet = 1 # 1: 서울, 2: 제주, 3: 부산
rc_no = list(range(1, 16))

## 경기 정보 불러오기
cnt = 0
for ym in rc_month:
    for rc in rc_no:
        try:
            detail_url = f'https://apis.data.go.kr/B551015/API4_2/raceResult_2?serviceKey=%2FpKtn0KFociUr9CM316rFT6o8sUrRD8yjSLuICPzxXarC8O%2FrUc1pjHc%2FCpuEwB5r4%2BimnNKYuJEjPKARyZCuw%3D%3D&pageNo=1&numOfRows=10000&meet={meet}&&rc_month={ym}&rc_no={rc}&_type=json'

            response = requests.get(detail_url)

            contents = response.text

            ## json을 DataFrame으로 변환하기 ##
            #문자열을 json으로 변경
            json_ob = json.loads(contents)

            # 필요한 내용만 꺼내기
            body = json_ob['response']['body']['items']['item']
        except Exception as e:
            print('e',e)
            continue

        # pandas import

        # Dataframe으로 만들기
        dataframe = pd.json_normalize(body)
        if 0 in dataframe.ord.unique():
            continue
        else:
            drop_n = np.sum(dataframe.ord.unique() > 90)
            if drop_n > 3:
                continue
            elif drop_n <= 3:
                dataframe = dataframe.drop(list(dataframe[dataframe['ord']>90].index))

            df_all = dataframe if cnt == 0 else pd.concat([df_all, dataframe], axis=0)
            cnt +=1

e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indices must be integers
e string indic

In [14]:
busan_info = list()
jeju_info = list()
seoul_info = list()
snj = list()

for c in df_all.columns:
    if ('buG' in c) or ('buS' in c) or ('bu_' in c) :
        busan_info.append(c)

    elif ('jeG' in c) or ('jeS' in c) or ('je_' in c) :
        jeju_info.append(c)
    
    elif ('seG' in c) or ('seS' in c) or ('se_' in c) :
        seoul_info.append(c)
    
    elif ('sj' in c):
        snj.append(c)
    

In [15]:
df = df_all.copy()
if meet==1:
    busan_info.extend(jeju_info)
    df.drop(busan_info, axis=1, inplace=True)
    print(df)

    age ageCond budam  buga1  buga2  buga3  chaksun1  chaksun2  chaksun3  \
0     3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
1     3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
2     3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
3     3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
4     3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
..  ...     ...   ...    ...    ...    ...       ...       ...       ...   
60    4    연령오픈   핸디캡      0      0      0  41250000  16500000  10500000   
61    6    연령오픈   핸디캡      0      0      0  41250000  16500000  10500000   
62    4    연령오픈   핸디캡      0      0      0  41250000  16500000  10500000   
63    5    연령오픈   핸디캡      0      0      0  41250000  16500000  10500000   
64    5    연령오픈   핸디캡      0      0      0  41250000  16500000  10500000   

    chaksun4  chaksun5  chulNo diffUnit  hrName     hrNo  ilsu jkName    jkNo  \
0    2

In [19]:
df.head()

age ageCond budam  buga1  buga2  buga3  chaksun1  chaksun2  chaksun3  \
0    3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
1    3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
2    3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
3    3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   
4    3    연령오픈   별정A      0      0      0   8800000   5500000   3740000   

   chaksun4  chaksun5  chulNo diffUnit  hrName     hrNo  ilsu jkName    jkNo  \
0   2420000   1540000       4        -  강서프린세스  0042371     4   안토니오  080576   
1   2420000   1540000       2        4   스틸더패션  0042584     4     먼로  080593   
2   2420000   1540000       1        4    농본바다  0043125     4    김효정  080578   
3   2420000   1540000       7       1¾    방탄여걸  0043346     4    김아현  080598   
4   2420000   1540000      12        3    팬시팬츠  0043204     4    송재철  080513   

  meet name  ord    owName    owNo  plcOdds prizeCond  rank  rating    rcDate  \
0   서울   한국    1  (주)디알엠씨티  114026   9999.9      R0~0  국6등급       0  20210129   
1   서울   한국    2       임동현  119010   9999.9      R0~0  국6등급       0  20210129   
2   서울   한국    3       김용래  065012   9999.9      R0~0  국6등급       0  20210129   
3   서울   한국    4       김태권  112044   9999.9      R0~0  국6등급       0  20210129   
4   서울   한국    5       윤지중  104100   9999.9      R0~0  국6등급       0  20210129   

  rcDay  rcDist rcName  rcNo  rcTime  seG1fAccTime  seG3fAccTime  \
0   금요일    1000     일반     1    60.6          47.8          23.8   
1   금요일    1000     일반     1    61.3          48.6          24.8   
2   금요일    1000     일반     1    62.0          48.7          24.3   
3   금요일    1000     일반     1    62.3          49.7          25.1   
4   금요일    1000     일반     1    62.8          48.9          24.3   

   seS1fAccTime  se_1cAccTime  se_2cAccTime  se_3cAccTime  se_4cAccTime sex  \
0          13.3           0.0           0.0          13.3          30.0   암   
1          13.9           0.0           0.0          13.9          30.9   암   
2          13.6           0.0           0.0          13.6          30.4   암   
3          13.9           0.0           0.0          13.9          31.5   암   
4          13.6           0.0           0.0          13.6          30.4   암   

   sjG1fOrd  sjG3fOrd  sjS1fOrd  sj_1cOrd  sj_2cOrd  sj_3cOrd  sj_4cOrd  \
0         1         1         1         0         0         1         1   
1         2         4         4         0         0         4         4   
2         3         2         2         0         0         2         2   
3         5         6         5         0         0         5         6   
4         4         3         3         0         0         3         3   

  trName    trNo     track weather  wgBudam      wgHr  winOdds  
0     토니  070265  다습 (11%)      맑음     54.5  515(-15)      1.6  
1    박병일  070163  다습 (11%)      맑음     54.5   500(-8)   9999.9  
2    배대선  070091  다습 (11%)      맑음     54.5  475(+10)      1.6  
3    이관호  070181  다습 (11%)      맑음     50.5   410(+2)   9999.9  
4    서홍수  070160  다습 (11%)      맑음     54.5     421()   9999.9

In [ ]:
non_valid = ['ageCond']

In [1]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.exceptions import ConvergenceWarning

import warnings

In [2]:
### Modeling 시 발생하는 Warning Message 무시하기 위한 과정 ###

warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
if not sys.warnoptions:
	warnings.simplefilter("ignore")			
	os.environ["PYTHONWARNINGS"] = "ignore"	

	warnings.filterwarnings('ignore', 'Solver terminated early.*')
	warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [ ]:

# data_split : 전체 data 분할 및 label 설정하는 함수
def data_split(input_df, y_col = 'label'):
    """ 
    input df : 학습에 사용할 입력 변수, 분류 범주가 모두 포함 된 Data Frame

    y_col : input_df 중 label에 해당하는 Column name
    """

    train_test_dict = dict()
    data_x = input_df.drop([y_col], axis=1) # input data 중 y_col에 해당하는 열 drop 하여 입력 변수에 해당하는 Data만 남김
    y = input_df[y_col].astype('int') # input data 중 y_col 에 해당하는 Data, y에 할당

    scaler = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(data_x.values, y.values, test_size=0.2, random_state=2020,shuffle=True)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

    train_test_dict['test_x'] = X_test
    train_test_dict['test_y'] = y_test
    train_test_dict['train_x'] = X_train
    train_test_dict['train_y'] = y_train

    return train_test_dict

train_test_dict = data_split(input_df=df, y_col='????')